<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #12: Análisis de sentimiento
`Fecha de entrega: Mayo 27, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Hacer análisis de sentimiento de los datos del anterior taller
- Columna con el puntaje numérico del sentimiento
- Pueden usar cualquiera de las librerias vistas en clase

In [1]:
import os
import pandas as pd
import json
import tweepy
import re 
import regex
import emoji
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import six
from google.cloud import translate_v2 as translate
from flair.models import TextClassifier
from flair.data import Sentence

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

vader = SentimentIntensityAnalyzer()

In [2]:
# Obtener variables de entorno desde sistema local
API_KEY = os.getenv('TWITTER_API_KEY')
API_SECRET_KEY = os.environ.get('TWITTER_API_SECRET_KEY')
BEARER_TOKEN =os.environ.get('TWITTER_BEARER_TOKEN')
ACCESS_TOKEN= os.environ.get('TWITTER_ACCESS_TOKEN')
ACCESS_TOKEN_SECRET= os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')


auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)
# obtener tweets con la misma temática del taller 11
tweets = tweepy.Cursor(api.search,
                       q=['Matarife -RT'],
                       lang='es',
                       geocode='4.3646,-74.0414,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(110)

# Objeto generador
t = [tweet for tweet in tweets]

t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
   
datos = pd.DataFrame(t_processed)
datos.columns = ['tweets', 'fecha', 'ciudad', 'localizacion_usuario', 'id', 'enlace']
datos.head()

,tweets,fecha,ciudad,localizacion_usuario,id,enlace
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...


In [3]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
datos[['texto_limpio', 'enlaces', 'hashtags', 'menciones']] = datos.apply(text_clean, axis=1, result_type='expand')
datos.head()



,tweets,fecha,ciudad,localizacion_usuario,id,enlace,texto_limpio,enlaces,hashtags,menciones
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...,El Matarife.,[],[],[@FVallejoQuotes]
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...,Jajajaja yo lo viví y soy antiuribista 100% s...,[],[],[@MargaritaRepo]
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...,"Si, que fue un raizal ... que quería desquit...",[],[],"[@Juanval121, @NoticiasCaracol]"
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...,PUES CLSRO CON TANTA PLATA DEL NARCOTRAFICO ...,[],[],"[@gabo180, @PAOLAHOLGUIN]"
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...,Los (mea culpa) ojo Colombia con esta nueva es...,[],[],[]


In [4]:
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

datos['emojis'] = datos['texto_limpio'].apply(lambda text: get_emojis(text))
datos.head()

,tweets,fecha,ciudad,localizacion_usuario,id,enlace,texto_limpio,enlaces,hashtags,menciones,emojis
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...,El Matarife.,[],[],[@FVallejoQuotes],[]
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...,Jajajaja yo lo viví y soy antiuribista 100% s...,[],[],[@MargaritaRepo],[]
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...,"Si, que fue un raizal ... que quería desquit...",[],[],"[@Juanval121, @NoticiasCaracol]",[]
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...,PUES CLSRO CON TANTA PLATA DEL NARCOTRAFICO ...,[],[],"[@gabo180, @PAOLAHOLGUIN]",[]
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...,Los (mea culpa) ojo Colombia con esta nueva es...,[],[],[],[]


In [5]:
# Definición de función de pre-procesamiento
stopwords_sp = stopwords.words('spanish')
stopwords_sp.extend(["https", "http", "co", "com", "edu","eu","Álvaro","Uribe", "Vélez", "Matarife", "u","û","q","k","w","a","p","pg","b","c","f","z","j",
                  "v","n","g","e","r","x","l","h","st","rt","lol","aa", "vélez","vélezel","álvaro","uribe"])

def preProcesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = texto.split() # Tokenizar
    texto = [palabra for palabra in texto if palabra not in stopwords_sp]
    texto = " ".join(texto)
    return texto

In [6]:
datos['pre_procesado'] = datos['texto_limpio'].apply(lambda x: preProcesado(x))
datos.head()

,tweets,fecha,ciudad,localizacion_usuario,id,enlace,texto_limpio,enlaces,hashtags,menciones,emojis,pre_procesado
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...,El Matarife.,[],[],[@FVallejoQuotes],[],matarife
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...,Jajajaja yo lo viví y soy antiuribista 100% s...,[],[],[@MargaritaRepo],[],jajajaja viví antiuribista sólo farsa asesino ...
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...,"Si, que fue un raizal ... que quería desquit...",[],[],"[@Juanval121, @NoticiasCaracol]",[],si raizal quería desquitarse armada reclamar t...
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...,PUES CLSRO CON TANTA PLATA DEL NARCOTRAFICO ...,[],[],"[@gabo180, @PAOLAHOLGUIN]",[],pues clsro tanta plata narcotrafico apadrinada...
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...,Los (mea culpa) ojo Colombia con esta nueva es...,[],[],[],[],mea culpa ojo colombia nueva estrategia tibios...


In [7]:
translate_client = translate.Client()

def traducir(texto, lenguaje):
    if isinstance(texto, six.binary_type):
        texto = texto.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    resultado = translate_client.translate(texto, target_language=lenguaje)
    return resultado['translatedText']

In [8]:
lenguaje = 'en'
datos['traducido'] = datos['pre_procesado'].apply(lambda x: traducir(x,lenguaje))
datos.head()

,tweets,fecha,ciudad,localizacion_usuario,id,enlace,texto_limpio,enlaces,hashtags,menciones,emojis,pre_procesado,traducido
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...,El Matarife.,[],[],[@FVallejoQuotes],[],matarife,matarife
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...,Jajajaja yo lo viví y soy antiuribista 100% s...,[],[],[@MargaritaRepo],[],jajajaja viví antiuribista sólo farsa asesino ...,hahahaha I lived anti-uribista only farce murd...
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...,"Si, que fue un raizal ... que quería desquit...",[],[],"[@Juanval121, @NoticiasCaracol]",[],si raizal quería desquitarse armada reclamar t...,"If Raizal wanted to get even with the army, cl..."
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...,PUES CLSRO CON TANTA PLATA DEL NARCOTRAFICO ...,[],[],"[@gabo180, @PAOLAHOLGUIN]",[],pues clsro tanta plata narcotrafico apadrinada...,"Well then, so much money for drug trafficking ..."
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...,Los (mea culpa) ojo Colombia con esta nueva es...,[],[],[],[],mea culpa ojo colombia nueva estrategia tibios...,mea culpa eye colombia new strategy lukewarm r...


In [9]:
classifier = TextClassifier.load('sentiment')

def sent_flair(texto):
    sentence = Sentence(texto)
    classifier.predict(sentence)
    result = sentence.labels[0]
    label = result.value
    score = result.score
    if label == 'POSITIVE':
        return score
    if label == 'NEGATIVE':
        return -1 * score
    return score

2021-05-26 22:37:16,394 loading file C:\Users\gabri\.flair\models\sentiment-en-mix-distillbert_4.pt


In [10]:
datos['flair_sentiment_es'] = datos['pre_procesado'].apply(lambda x: sent_flair(x))
datos['vader_sentiment_es'] = datos['pre_procesado'].apply(lambda x:  vader.polarity_scores(x)['compound'])
datos['textblob_sentiment_es'] = datos['pre_procesado'].apply(lambda x: TextBlob(x).sentiment.polarity)
datos['textblob_sentiment_en'] = datos['traducido'].apply(lambda x: TextBlob(x).sentiment.polarity)
datos['vader_sentiment_en'] = datos['traducido'].apply(lambda x: vader.polarity_scores(x)['compound'])
datos['flair_sentiment_en'] = datos['traducido'].apply(lambda x: sent_flair(x))
datos.head()
datos.head()

,tweets,fecha,ciudad,localizacion_usuario,id,enlace,texto_limpio,enlaces,hashtags,menciones,emojis,pre_procesado,traducido,flair_sentiment_es,vader_sentiment_es,textblob_sentiment_es,textblob_sentiment_en,vader_sentiment_en,flair_sentiment_en
0,@FVallejoQuotes El Matarife.,2021-05-27 03:25:25,Bogotá,"Manizales, Colombia",1397755795060760576,https://twitter.com/i/web/stuatus/139775579506...,El Matarife.,[],[],[@FVallejoQuotes],[],matarife,matarife,0.940072,0.0,0.0,0.000000,0.0000,0.940072
1,@MargaritaRepo Jajajaja yo lo viví y soy antiu...,2021-05-27 03:24:51,Bogotá,"Bogotá, D.C., Colombia",1397755652727005186,https://twitter.com/i/web/stuatus/139775565272...,Jajajaja yo lo viví y soy antiuribista 100% s...,[],[],[@MargaritaRepo],[],jajajaja viví antiuribista sólo farsa asesino ...,hahahaha I lived anti-uribista only farce murd...,-0.960450,0.0,0.0,0.100000,-0.2732,-0.995891
2,"@Juanval121 @NoticiasCaracol Si, que fue un ra...",2021-05-27 03:22:21,Bogotá,"Bogotá, D.C., Colombia",1397755022348361730,https://twitter.com/i/web/stuatus/139775502234...,"Si, que fue un raizal ... que quería desquit...",[],[],"[@Juanval121, @NoticiasCaracol]",[],si raizal quería desquitarse armada reclamar t...,"If Raizal wanted to get even with the army, cl...",0.992382,0.0,0.0,0.200000,0.5574,-0.986793
3,@gabo180 PUES CLSRO @PAOLAHOLGUIN CON TANTA PL...,2021-05-27 03:20:14,Bogotá,"Bogotá, D.C., Colombia",1397754492171472897,https://twitter.com/i/web/stuatus/139775449217...,PUES CLSRO CON TANTA PLATA DEL NARCOTRAFICO ...,[],[],"[@gabo180, @PAOLAHOLGUIN]",[],pues clsro tanta plata narcotrafico apadrinada...,"Well then, so much money for drug trafficking ...",0.571968,0.0,0.0,0.200000,0.2732,-0.998634
4,Los (mea culpa) ojo Colombia con esta nueva es...,2021-05-27 03:17:08,Bogotá,BOGOTÁ,1397753712429473794,https://twitter.com/i/web/stuatus/139775371242...,Los (mea culpa) ojo Colombia con esta nueva es...,[],[],[],[],mea culpa ojo colombia nueva estrategia tibios...,mea culpa eye colombia new strategy lukewarm r...,0.998729,0.0,0.0,0.107359,-0.2500,-0.999603


###  `[10 pts]` Punto 2: ¿Cuál es el tuit más positivo y más negativo?

In [15]:
dictLims = {"maximo_vader_es":datos['vader_sentiment_es'].max(), 
            "minimo_vader_es":datos['vader_sentiment_es'].min(),
            "maximo_textblob_es":datos['textblob_sentiment_es'].max(),
            "minimo_textblob_es":datos['textblob_sentiment_es'].min(),
            "maximo_flair_es":datos['flair_sentiment_es'].max(),
            "minimo_flair_es":datos['flair_sentiment_es'].min(),
            "maximo_vader_en":datos['vader_sentiment_en'].max(), 
            "minimo_vader_en":datos['vader_sentiment_en'].min(),
            "maximo_textblob_en":datos['textblob_sentiment_en'].max(),
            "minimo_textblob_en":datos['textblob_sentiment_en'].min(),
            "maximo_flair_en":datos['flair_sentiment_en'].max(),
            "minimo_flair_en":datos['flair_sentiment_en'].min()            
           }

minMaxDatos = pd.DataFrame.from_dict(dictLims, orient='index', columns=['valor'])
minMaxDatos


,valor
maximo_vader_es,0.636900
minimo_vader_es,-0.599400
maximo_textblob_es,0.600000
minimo_textblob_es,-0.500000
maximo_flair_es,0.999712
minimo_flair_es,-0.999941
maximo_vader_en,0.911800
minimo_vader_en,-0.962300
maximo_textblob_en,0.800000
minimo_textblob_en,-1.000000


###  `[10 pts]` Punto 3: ¿Cuántos tuits positivos/negativos hay en sus datos?

In [76]:
# Flitros Flair en español
positivosFlairEs = datos[['flair_sentiment_es']][(datos["flair_sentiment_es"] >0.0)]
negativosFlairEs = datos[['flair_sentiment_es']][(datos["flair_sentiment_es"] <0.0)]
neutrosFlairEs = datos[['flair_sentiment_es']][(datos["flair_sentiment_es"] ==0.0)]
# Filtros Vader en español
positivosVaderEs = datos[['vader_sentiment_es']][(datos["vader_sentiment_es"] >0.0)]
negativosVaderEs = datos[['vader_sentiment_es']][(datos["vader_sentiment_es"] <0.0)]
neutrosVaderEs = datos[['vader_sentiment_es']][(datos["vader_sentiment_es"] ==0.0)]
# Filtros Textblob en español
positivosTextblobEs = datos[['textblob_sentiment_es']][(datos["textblob_sentiment_es"] >0.0)]
negativosTextblobEs = datos[['textblob_sentiment_es']][(datos["textblob_sentiment_es"] <0.0)]
neutrosTextblobEs = datos[['textblob_sentiment_es']][(datos["textblob_sentiment_es"] ==0.0)]
#Filtros flair en inglés
positivosFlairEn = datos[['flair_sentiment_en']][(datos["flair_sentiment_en"] >0.0)]
negativosFlairEn = datos[['flair_sentiment_en']][(datos["flair_sentiment_en"] <0.0)]
neutrosFlairEn = datos[['flair_sentiment_en']][(datos["flair_sentiment_en"] ==0.0)]
# Filtros Vader en inglés
positivosVaderEn = datos[['vader_sentiment_en']][(datos["vader_sentiment_en"] >0.0)]
negativosVaderEn = datos[['vader_sentiment_en']][(datos["vader_sentiment_en"] <0.0)]
neutrosVaderEn = datos[['vader_sentiment_en']][(datos["vader_sentiment_en"] ==0.0)]
# Filtros Textblob en inglés
positivosTextblobEn = datos[['textblob_sentiment_en']][(datos["textblob_sentiment_en"] >0.0)]
negativosTextblobEn = datos[['textblob_sentiment_en']][(datos["textblob_sentiment_en"] <0.0)]
neutrosTextblobEn = datos[['textblob_sentiment_en']][(datos["textblob_sentiment_en"] ==0.0)]

# conteos flair español
conteoPositivosFlairEs = positivosFlairEs.count()
conteoNegativosFlairEs = negativosFlairEs.count()
conteoNeutrosFlairEs = neutrosFlairEs.count()

# conteos vader español
conteoPositivosVaderEs = positivosVaderEs.count()
conteoNegativosVaderEs = negativosVaderEs.count()
conteoNeutrosVaderEs = neutrosVaderEs.count()

# conteos textblob español
conteoPositivosTextblobEs = positivosTextblobEs.count()
conteoNegativosTextblobEs = negativosTextblobEs.count()
conteoNeutrosTextblobEs = neutrosTextblobEs.count()

# conteos flair inglés
conteoPositivosFlairEn = positivosFlairEn.count()
conteoNegativosFlairEn = negativosFlairEn.count()
conteoNeutrosFlairEn = neutrosFlairEn.count()

# conteos vader inglés
conteoPositivosVaderEn = positivosVaderEn.count()
conteoNegativosVaderEn = negativosVaderEn.count()
conteoNeutrosVaderEn = neutrosVaderEn.count()

# conteos textblob español
conteoPositivosTextblobEn = positivosTextblobEn.count()
conteoNegativosTextblobEn = negativosTextblobEn.count()
conteoNeutrosTextblobEn = neutrosTextblobEn.count()


conteosData = pd.concat([conteoPositivosFlairEs, conteoNegativosFlairEs, conteoNeutrosFlairEs,
                        conteoPositivosVaderEs, conteoNegativosVaderEs, conteoNeutrosVaderEs,
                        conteoPositivosTextblobEs, conteoNegativosTextblobEs, conteoNeutrosTextblobEs,
                        conteoPositivosFlairEn, conteoNegativosFlairEn, conteoNeutrosFlairEn,
                        conteoPositivosVaderEn, conteoNegativosVaderEn, conteoNeutrosVaderEn,
                        conteoPositivosTextblobEn, conteoNegativosTextblobEn, conteoNeutrosTextblobEn], axis= 0)

listaValores = ['Positivo flair (es)', 'Negativo flair (es)', 'Neutro flair (es)',
               'Positivo vader (es)', 'Negativo vader (es)', 'Neutro vader (es)',
               'Positivo textblob (es)', 'Negativo textblob (es)', 'Neutro textblob (es)',
               'Positivo flair (en)', 'Negativo flair (en)', 'Neutro flair (en)',
               'Positivo vader (en)', 'Negativo vader (en)', 'Neutro vader (en)',
               'Positivo textblob (en)', 'Negativo textblob (en)', 'Neutro textblob (en)']
conteosData =pd.DataFrame(conteosData).reset_index()

conteosData.columns = ['Atributo','conteo']


for i in (range(len(conteosData))):
    conteosData.loc[i:i,'Atributo'] = listaValores[i]

conteosData

,Atributo,conteo
0,Positivo flair (es),61
1,Negativo flair (es),49
2,Neutro flair (es),0
3,Positivo vader (es),4
4,Negativo vader (es),4
5,Neutro vader (es),102
6,Positivo textblob (es),3
7,Negativo textblob (es),5
8,Neutro textblob (es),102
9,Positivo flair (en),30


###  `[5 pts]` : Punto 4: Una visualización sobre sus datos

In [78]:
import plotly.express as px

fig = px.bar(conteosData, x="Atributo", y="conteo", color="Atributo", title="Conteo de tweets")
fig.show()